In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
%matplotlib inline
import numpy as np
import seaborn as sns

In [3]:
# Historical FIFA rankings dataframe
historical_rankings = pd.read_csv('fifa_ranking.csv')
# Historical match between 2 teams dataframe
historical_matches = pd.read_csv('results.csv')
world_cup = pd.read_csv('World Cup 2018 Dataset.csv')

In [4]:
historical_rankings = historical_rankings.replace({'IR Iran': 'Iran'})
historical_matches = historical_matches.replace({'Germany DR': 'Germany', 'China': 'China PR'})
world_cup = world_cup.replace({"IRAN": "Iran", 
                               "Costarica": "Costa Rica", 
                               "Porugal": "Portugal", 
                               "Columbia": "Colombia", 
                               "Korea" : "Korea Republic"})
world_cup = world_cup.set_index('Team')

In [5]:
print("Historical FIFA Rankings:")
print(historical_rankings.info())
print("--------------------------------")
print("Historical matches")
print(historical_matches.info())

Historical FIFA Rankings:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793 non-null float64
last_year_avg              57793 non-null float64
last_year_avg_weighted     57793 non-null float64
two_year_ago_avg           57793 non-null float64
two_year_ago_weighted      57793 non-null float64
three_year_ago_avg         57793 non-null float64
three_year_ago_weighted    57793 non-null float64
confederation              57793 non-null object
rank_date                  57793 non-null object
dtypes: float64(9), int64(3), object(4)
memory usage: 7.1+ MB
None
--------

In [6]:
historical_rankings = historical_rankings.loc[:,['rank', 'country_full','country_abrv', 'total_points', 'confederation', 'rank_date']]
historical_rankings['rank_date'] = pd.to_datetime(historical_rankings['rank_date'])
historical_rankings = historical_rankings.set_index(['rank_date'])\
            .groupby(['country_full'], group_keys=False)\
            .resample('D').first()\
            .fillna(method='ffill')\
            .reset_index()
historical_rankings.head()

,rank_date,rank,country_full,country_abrv,total_points,confederation
0,2003-01-15,204.0,Afghanistan,AFG,0.0,AFC
1,2003-01-16,204.0,Afghanistan,AFG,0.0,AFC
2,2003-01-17,204.0,Afghanistan,AFG,0.0,AFC
3,2003-01-18,204.0,Afghanistan,AFG,0.0,AFC
4,2003-01-19,204.0,Afghanistan,AFG,0.0,AFC


In [7]:

historical_rankings_by_team = historical_rankings.groupby([historical_rankings.rank_date.dt.year,historical_rankings.country_full]).mean()
historical_rankings_by_team = historical_rankings_by_team.reset_index()

In [8]:
print("Top 10 Averaged World Rank since 1993")
historical_rankings_by_team.groupby(['country_full']).mean().sort_values(by=['rank'])['rank'][:10]

Top 10 Averaged World Rank since 1993


country_full
Brazil          3.085171
Germany         4.960291
Spain           5.485745
Argentina       5.690997
Italy           8.218103
France          9.087425
Netherlands     9.136396
England        10.826872
Portugal       11.731863
Mexico         14.675684
Name: rank, dtype: float64

In [10]:
print("Top 32 Teams Averaged World Rank That Failed to Qualify to World Cup 2018")
top_32 = historical_rankings_by_team.groupby(['country_full']).mean().sort_values(by=['rank'])['rank'][:32].reset_index()
not_in_world_cup_2018 = top_32[top_32['country_full'].isin(world_cup.index.unique())]
print(not_in_world_cup_2018)

Top 32 Teams Averaged World Rank That Failed to Qualify to World Cup 2018
   country_full       rank
0        Brazil   3.085171
1       Germany   4.960291
2         Spain   5.485745
3     Argentina   5.690997
5        France   9.087425
7       England  10.826872
8      Portugal  11.731863
9        Mexico  14.675684
12      Denmark  19.559398
13     Colombia  20.144352
15       Sweden  21.587842
16       Russia  23.254707
17      Uruguay  23.343179
19      Croatia  24.735628
21      Belgium  29.846761
22  Switzerland  30.459317
27        Egypt  33.781895
28       Serbia  34.964922
29        Japan  35.301610
